In [1]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample

In [7]:
val_rep={0:0.4,0.01:0.1}
URM_path2="../data/interactions_and_impressions_v4.csv"
URM_2,ICM_all=Reader.load_URM_and_ICM_items(URM_path2,"../data/data_ICM_type.csv", URM_cols=["UserID","ItemID","Interaction"], URM_dtye={0: int, 1: int, 2: float}, ICM_cols=["ItemID","FeatureID","oo"],URM_values_to_replace={0:0.2},matrix_format="ICM")
cutoff_list=[ 10]
_,URM2,URM_train,URM_train2,URM_test,_=Reader.split_train_validation_double(URM_path2=URM_path2,values_to_replace=val_rep,matrix_format="total",URM2=URM_2,URM2_cols=["UserID","ItemID","Data"])


URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)
URM_train2, URM_validation2 = split_train_in_two_percentage_global_sample(URM_train2, train_percentage = 0.8)

stacked_ICM = Reader.combine(ICM=ICM_all, URM = URM_train2)
stacked_URM=stacked_ICM.T
stacked_URM

Number of tags	 5, Number of item-tag tuples 23091
Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 27968
Unique FeatureID in the URM are 5
EvaluatorHoldout: Ignoring 765 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 323 ( 0.8%) Users that have less than 1 test interactions


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 27968, expected 24450.

In [6]:
URM_2

<41629x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 2928612 stored elements in Compressed Sparse Row format>

In [10]:

#FACTORIZED SPLIT
URM_all,ICM_all=Reader.load_URM_and_ICM_items("../data/interactions_and_impressions.csv","../data/data_ICM_type.csv", ICM_cols=["ItemID","FeatureID","oo"],
                                               URM_values_to_replace={0:0.2})
cutoff_list=[ 10]



URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)
'''
profile_length = np.ediff1d(URM_all.indptr)
block_size = int(len(profile_length)*0.25)

start_lower = 0
end_lower = 3*block_size
end_higher = len(profile_length)
sorted_users = np.argsort(profile_length)

users_in_lower = set(sorted_users[0:end_lower])
'''
#ICM_train, ICM_test = split_train_in_two_percentage_global_sample(ICM_all ,train_percentage=0.80)

stacked_ICM = Reader.combine(ICM=ICM_all, URM = URM_train)
stacked_URM=stacked_ICM.T
stacked_URM

Number of tags	 5, Number of item-tag tuples 23091
Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 27968
Unique FeatureID in the URM are 5
EvaluatorHoldout: Ignoring 738 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 282 ( 0.7%) Users that have less than 1 test interactions


<41634x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1018061 stored elements in Compressed Sparse Column format>

In [18]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemUserHybridKNNRecommender = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemUserHybridKNNRecommender.fit()

result_df, _ = evaluator_test.evaluateRecommender(recommender_ItemUserHybridKNNRecommender)
result_df

ItemUserHybridKNNRecommender: URM Detected 5 ( 0.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 5 ( 0.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 5 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 5077.90 column/sec. Elapsed time 4.83 sec
Similarity column 41629 (100.0%), 2523.76 column/sec. Elapsed time 16.49 sec
[[-0.1743116  -0.2002699  -0.20286919 ... -0.21314976 -0.05559506
  -0.28413147]
 [-0.28413147 -0.24397035 -0.28413147 ... -0.28413147 -0.04739182
  -0.28413147]
 [ 0.39848518  0.76796615  0.3610461  ... -0.08837295 -0.25501063
  -0.04199498]
 ...
 [-0.28413147 -0.28413147 -0.28413147 ... -0.28413147 -0.28413147
  -0.28413147]
 [-0.08854178  0.07388707 -0.07438271 ... -0.20830062 -0.28413147
  -0.0764095 ]
 [-0.20282905  0.51242477  0.08343274 ... -0.28413147 -0.28413147
  -0.28413147]]
[[-0.03266129 -0.01679668 -0.02099377 ... -0.0262429  -0.00329354
  -0.03266129]
 [-0.03266129 -0.03266129 -0.03266129 ... -0.03

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[-0.37676755 -0.21202704 -0.37676755 ... -0.30694005 -0.37676755
  -0.37676755]
 [-0.0798706   0.450665    0.29188398 ... -0.08346202 -0.22176611
  -0.18025483]
 [-0.30991307 -0.04816767 -0.30219328 ... -0.37676755 -0.37676755
  -0.37676755]
 ...
 [-0.30066663 -0.23294656 -0.2688655  ... -0.37676755 -0.37676755
  -0.3119811 ]
 [-0.37676755 -0.18142591 -0.28408188 ... -0.37676755 -0.37676755
  -0.37676755]
 [-0.26353678 -0.310339   -0.3508627  ... -0.31971657 -0.37676755
  -0.33382246]]
[[-0.0429483  -0.01750922 -0.0429483  ... -0.0429483  -0.0429483
  -0.0429483 ]
 [-0.0429483  -0.0087974  -0.0429483  ... -0.0429483  -0.0429483
  -0.0429483 ]
 [-0.0429483  -0.02936254 -0.0429483  ... -0.0429483  -0.0429483
  -0.0429483 ]
 ...
 [-0.0429483  -0.03004062 -0.01547754 ... -0.0429483  -0.0429483
  -0.0429483 ]
 [-0.0429483  -0.02030473 -0.0429483  ... -0.0429483  -0.0429483
  -0.0429483 ]
 [-0.0137227  -0.0429483  -0.0429483  ... -0.03126813 -0.0429483
  -0.0429483 ]]
[[-0.39676157 -0.18094

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/Evaluator.py:368: RuntimeWarning: invalid value encountered in double_scalars
  results_current_cutoff[EvaluatorMetrics.NDCG.value]                 += ndcg(recommended_items_current_cutoff, relevant_items, relevance=self.get_user_test_ratings(test_user), at=cutoff)


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.044347,0.076804,0.072376,0.020916,0.03547,0.141521,NaN,0.054996,0.299513,0.170944,...,0.96219,0.288189,0.96219,0.054737,9.5748,0.993406,0.125549,0.726572,3.395432,0.51646


In [17]:

from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender

rp3betaCBF_recommender = RP3betaCBFRecommender(
    URM_train=URM_train2,
    ICM_train=ICM_all,
    verbose=False
)

rp3betaCBF_recommender.fit(
    topK=int(175.1628484087545),
    alpha=0.42304737831676245,
    beta=0.256647894170121,
    implicit=False
)

result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
result_df["MAP"]


ValueError: dimension mismatch

In [19]:
result_df, _ = evaluator_test.evaluateRecommender(rp3betaCBF_recommender)
result_df["MAP"]

ValueError: dimension mismatch

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#1
import numpy
import itertools
from tqdm.notebook import tqdm
# GRID SEARCH RP3BETA
best=0
config={
    "k":0,
    "a":0,
    "b":0

}
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender
URM_all=Reader.read_train_csr(values_to_replace={0:0.2})
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

rp3betaCBF_recommender = RP3betaRecommender(
    URM_train=URM_train,
    verbose=False
)

TopkList=numpy.arange(100, 800, 50)
aList=numpy.arange(0.1, 0.8, 0.01)
bList=numpy.arange(0.1, 0.8, 0.01)
for x in tqdm(itertools.product(TopkList,aList,bList),total=int(len(TopkList)*len(aList)*len(bList)), position=0, leave=True):
    k=x[0]
    a=x[1]
    b=x[2]
    rp3betaCBF_recommender.fit(
        topK=k,
        alpha=a,
        beta=b,
        implicit=False
    )

    result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        config={
            "k":k,
            "a":a,
            "b":b
        }
        print("NEW BEST: "+str(best))
        print("Config:"+str(config))

best,config

EvaluatorHoldout: Processed 40891 (100.0%) in 37.99 sec. Users per second: 1076


In [ ]:
best,config